In [ ]:
#Import Statements Here
import numpy as np
import pandas as pd
from statistics import mean
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as skl
from sklearn import metrics
from scipy import stats
rng = np.random.default_rng()
from sklearn import linear_model
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
#from venn import venn

In [ ]:
import os
import subprocess

In [ ]:
## This snippet assumes that you run setup first

## This code lists objects in your Google Bucket

## Get the bucket name
#my_bucket = os.getenv('WORKSPACE_BUCKET')

## List objects in the bucket
#print(subprocess.check_output(f"gsutil ls -r {my_bucket}", shell=True).decode('utf-8'))




In [ ]:
def getFromBucket(name_of_file_in_bucket):
    # get the bucket name
    my_bucket = os.getenv('WORKSPACE_BUCKET')

    # copy csv file from the bucket to the current working space
    os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

    print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
    # save dataframe in a csv file in the same workspace as the notebook
    my_dataframe = pd.read_csv(name_of_file_in_bucket)
    return my_dataframe


In [ ]:
def storeInBucket(name_of_file_loc, a = 'dataFrame'):
    # get the bucket name
    
    # This snippet assumes you run setup first

    # This code saves your dataframe into a csv file in a "data" folder in Google Bucket

    # Replace df with THE NAME OF YOUR DATAFRAME
    if (type(name_of_file_loc)== type('string')):
        my_dataframe = pd.read_csv(name_of_file_loc)
        destination_filename = name_of_file_loc
    else:
        my_dataframe = name_of_file_loc
        destination_filename = a

    # Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
    

    ########################################################################
    ##
    ################# DON'T CHANGE FROM HERE ###############################
    ##
    ########################################################################

    # save dataframe in a csv file in the same workspace as the notebook
    my_dataframe.to_csv(destination_filename, index=False)

    # get the bucket name
    my_bucket = os.getenv('WORKSPACE_BUCKET')

    # copy csv file to the bucket
    args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
    output = subprocess.run(args, capture_output=True)

    # print output from gsutil
    output.stderr
    

## Gather Mobile Device Data

In [ ]:
InnerMobileData = getFromBucket('InnerJoinMarch16MobileDevice.csv')
OuterMobileData = getFromBucket('InnerJoinMarch16MobileDevice.csv')

# Genetic Data

## Genetic Data Breakdown

In [ ]:
#storeInBucket("Jan2023GeneticData.csv")

GeneticDataDf = getFromBucket("GeneticDataDfAugust14.csv")
GeneticDataDf

In [ ]:
DistributionDf = pd.DataFrame(columns = ['Genetic Variant','Percent Expression'])
for a in range(1,37):
    Gene = GeneticDataDf.columns[a]
    Proportion = len(GeneticDataDf[GeneticDataDf.iloc[:,a]>0])/4803
    df2 = {'Genetic Variant':Gene,'Percent Expression':Proportion}
    #DistributionDf = DistributionDf.append(df2, ignore_index=True)

In [ ]:
#DistributionDf = DistributionDf.drop(25)
#DistributionDf

In [ ]:
CorrDf=GeneticDataDf.corr().drop(['chr16:89196249:G:GGCCGGCTGCCTGGCCGCCCTGCCGGCCGTGCAGGGTGTCCAGGA_GGCCGGCTGCCTGGCCGCCCTGCCGGCCGTGCAGGGTGTCCAGGA'])
CorrDf  = CorrDf.drop(['chr16:89196249:G:GGCCGGCTGCCTGGCCGCCCTGCCGGCCGTGCAGGGTGTCCAGGA_GGCCGGCTGCCTGGCCGCCCTGCCGGCCGTGCAGGGTGTCCAGGA'], axis =1)
CorrDf

In [ ]:
for a in CorrDf.index:
    for b in CorrDf.columns:
        if (a != b):
            if (abs(CorrDf.loc[a,b]) > 0.75):
                print(a + " vs " + b + " = " + str(CorrDf.loc[a,b]))

In [ ]:
#print(GeneticDataDf.columns)
    


In [ ]:
#Remove the extra genetic Variables, and any variables that don't show any representation in the population
ToDropList = ['chr16:89196249:G:GGCCGGCTGCCTGGCCGCCCTGCCGGCCGTGCAGGGTGTCCAGGA_GGCCGGCTGCCTGGCCGCCCTGCCGGCCGTGCAGGGTGTCCAGGA',
             'chr3:45859597:C:T_T','chr3:101705614:T:C_C', 'chr3:101705614:TG:T_T' , 'chr6:31153649:G:A_A','chr6:41534945:A:C_C',
              'chr11:1219991:G:A_A', 'chr11:1219991:G:C_C', 'chr12:132564254:T:G_G','chr16:89196249:G:T_T', 'chr17:45707983:T:G_G',
              'chr21:33252612:A:G_A']

GeneticDataDf = GeneticDataDf.drop(ToDropList , axis = 1)
#GeneticDataDf 

In [ ]:
CorrDf=GeneticDataDf.corr()
CorrDf

In [ ]:
for a in CorrDf.index:
    for b in CorrDf.columns:
        if (a != b):
            if (abs(CorrDf.loc[a,b]) > 0.75):
                print(a + " vs " + b + " = " + str(CorrDf.loc[a,b]))

### Add in n3c cohort

In [ ]:
n3cCohort = getFromBucket('n3c_aou_cohort_ft_May_23.csv')

## Adding in EHR, survey data and Long Covid Status

In [ ]:
#Load in Long Covid EHR data from google bucket
LongCovidEHRDf = getFromBucket('n3c_aou_cohort.csv')
#for a in LongCovidEHRDf.columns:
    #print(a)

In [ ]:
#Load in Survey Data from google bucket
SurveyDataDF = getFromBucket('SurveyDataMarch2023.csv')
SurveyDataDF= SurveyDataDF.sort_values('person_id')
drop_list = ['Race: What Race Ethnicity', 'The Basics: Sexual Orientation',
             'Home Own: Current Home Own','Income: Annual Income','Education Level: Highest Grade','Living Situation: Stable House Concern']
SurveyDataDF = SurveyDataDF.drop(drop_list, axis=1)
SurveyDataDF

In [ ]:
#Load in Genetic Data from google bucket
#currently this data is loaded in from the same notebook, so this cell is irrelevant until we do our sorting later on

#GeneticDataDf = getFromBucket('GeneticDataFeb2023.csv')
#GeneticDataDf= GeneticDataDf.sort_values('person_id')
#GeneticDataDf

### Select Variables from Genetic data, Survey data, and EHR data to include in the model 

In [ ]:
def selectVariables(df, variables):
    returnDf = df.loc[:, variables]
    returnDf=returnDf.sort_values('person_id', ignore_index=True)
    return returnDf

In [ ]:
#We select the data variables that we want from our EHR data added to our dataframe here

EHRAppendDf = selectVariables( LongCovidEHRDf, ['y_pred','person_id','long_covid'] )


In [ ]:
#We select the data variables that we want from our survey data added to our dataframe here

nonDuplicatesVariables = []

for a in SurveyDataDF.columns:
    if a in GeneticDataDf.columns:
        print(a)
    else: 
        nonDuplicatesVariables = nonDuplicatesVariables+ [a]
        
nonDuplicatesVariables = nonDuplicatesVariables+ ['person_id']

In [ ]:
surveyAppendDf = selectVariables(SurveyDataDF, nonDuplicatesVariables)

### Remove non-numeric Values

In [ ]:
def removeNonNumeric(df):
    #df = df[df['long_covid'] > - 1]
    df = df.replace('NaN', np.NaN)
    df = df.replace('nan', np.NaN)
    for a in df.columns:
    #columnDf = trialDf[a]
        columnMean = df.loc[df[a].notna() ,a].mean()
    #print(a)
    #print(columnMean)
        df[a] = df[a].fillna(value = columnMean)
    returnDf = df
    return returnDf

In [ ]:
#We select the data variables that we want from our genetic data added to our dataframe here
GeneticDataDf = removeNonNumeric(GeneticDataDf)
geneAppendDf = selectVariables(GeneticDataDf, GeneticDataDf.columns)

In [ ]:
#GeneticDataDf

### Create a gold standard cohort

In [ ]:
#Here we are going to join just survey data and mobile device data

def joinMobileDeviceWithSurveyData(mobileDf, surveyDf, mergeType = 'inner'):
    
    returnDf = pd.merge(left = mobileDf, right = surveyDf, how = mergeType, left_on = 'person_id', right_on='person_id')
    
    return returnDf

In [ ]:
def joinMobileDeviceWithGeneticData(mobileDf, geneAppendDf, mergeType = 'inner'):
    
    returnDf = pd.merge(left = mobileDf, right = geneAppendDf, how = mergeType, left_on = 'person_id', right_on='person_id')
    
    return returnDf

In [ ]:
innerJoinSurveyMobileDevice = joinMobileDeviceWithSurveyData(InnerMobileData, surveyAppendDf, mergeType = 'outer')
innerJoinAllData = joinMobileDeviceWithGeneticData(innerJoinSurveyMobileDevice, geneAppendDf, mergeType = 'inner')
innerJoinAllData

In [ ]:
goldStandardPersons = innerJoinAllData['person_id']

### Combine data from all data sources by person_id

In [ ]:
####Start here and go down######
#We want to include the alterations to the survey data df
#Make a new machineLearningDf based on survey, genetic, and mobile device data

In [ ]:
#takes a bunch of dataframes, and joins them by the joining variable
    #duplicate columns will be dropped at the end, 
    #so if there are still duplicate column names, that means some of the columns don't match
def combineData(dataFrameList, joinVariable):
    joiningSet = dataFrameList[0].loc[:,joinVariable]
    
    
    #Now redefine joiningSet by going through each dataframe in the list 
    #and redefine as only the joiningVariables that are both in that dataframe, and the previous joiningSet
    for df in dataFrameList:
        joiningSet = df.loc[df[joinVariable].isin(joiningSet),joinVariable]
    
    
    #now that we have a joiningSet that includes only the common part of every data set, 
    #we need to restrict each data frame according to that set
    
    cleanedDfList = []
    for df in dataFrameList:
        cleanedDf = df.loc[df[joinVariable].isin(joiningSet),:].sort_values(joinVariable, ignore_index=True)
        cleanedDfList = cleanedDfList + [cleanedDf]
        
    #now join them all together with a concat statement and remove duplicate columns
    returnDf=pd.concat(cleanedDfList,axis=1)
    returnDf = returnDf.T.drop_duplicates().T
    return returnDf
        

In [ ]:
machineLearningDf = combineData([EHRAppendDf, surveyAppendDf, geneAppendDf], 'person_id')
machineLearningDf= removeNonNumeric(machineLearningDf)

### Simplifying assumption

In [ ]:
# Here is the simplifying assumption that takes the data and changes the long_covid valuies of high y_pred patients

#machineLearningDf
def simplifyingAssumption(df, p=0.9):
    for i in range(0,len(df)):
        if (df.loc[i,'y_pred'] >= p):
            df.loc[i,'long_covid']=1
        
    return df
        


In [ ]:
#machineLearningDf = simplifyingAssumption(machineLearningDf)
#We want to apply the simplifying assumption to only test and train, not gold standard set

In [ ]:
def removeNonNumeric(df):
    df = df[df['long_covid'] > - 1]
    df = df.replace('NaN', np.NaN)
    df = df.replace('nan', np.NaN)
    for a in df.columns:
    #columnDf = trialDf[a]
        columnMean = df.loc[df[a].notna() ,a].mean()
    #print(a)
    #print(columnMean)
        df[a] = df[a].fillna(value = columnMean)
    returnDf = df
    return returnDf    


In [ ]:
machineLearningDf = removeNonNumeric(machineLearningDf)

### Adjacent: Combine data from Survey data and Mobile device data in a seperate dataframe

In [ ]:
#here we make a dataframe of just survey data and mobile device data with TONS of empty values
#We are going to apply this to a different algorithm that can handle these empty values
outerJoinSurveyMobileDevice = joinMobileDeviceWithSurveyData(OuterMobileData, surveyAppendDf, mergeType = 'outer')
#outerJoinSurveyMobileDevice= simplifyingAssumption(outerJoinSurveyMobileDevice)
outerJoinSurveyMobileDevice = outerJoinSurveyMobileDevice.drop('min_covid_dt', axis=1)
outerJoinSurveyMobileDevice = removeNonNumeric(outerJoinSurveyMobileDevice)

In [ ]:
#outerJoinSurveyMobileDevice = outerJoinSurveyMobileDevice.drop(['Race: What Race Ethnicity', 'The Basics: Sexual Orientation'], axis=1)
#outerJoinSurveyMobileDevice.columns

### Control model Data Frame with ALL data types

In [ ]:
#Append EHR, Genetic, Survey, and Mobile Device Data into a big dataframe with lots of wholes

#EHR Dataframe
LongCovidEHRDf = LongCovidEHRDf.sort_values('person_id')

#Survey Dataframe
SurveyDataDF = SurveyDataDF.sort_values('person_id')

#Genetic Dataframe
GeneticDataDf = GeneticDataDf.sort_values('person_id')

#Mobile Device Dataframe
OuterMobileData = OuterMobileData.sort_values('person_id')

#Use Existing Join function to join all the data
step1 = joinMobileDeviceWithSurveyData(LongCovidEHRDf, SurveyDataDF, mergeType = 'outer')
step2 = joinMobileDeviceWithSurveyData(step1, GeneticDataDf, mergeType = 'outer')
step3 = joinMobileDeviceWithSurveyData(step2, OuterMobileData, mergeType = 'outer')

ControlModelAllDataDf = step3

In [ ]:
ControlModelAllDataDf = ControlModelAllDataDf.drop(
    ['index', 'long_covid_y','min_long_covid_date','y_pred_y']
    , axis = 1)


#Rename long_covid_x as long_covid
ControlModelAllDataDf = ControlModelAllDataDf.rename(columns={'long_covid_x':'long_covid'})
ControlModelAllDataDf = ControlModelAllDataDf.rename(columns={'y_pred_x':'y_pred'})

In [ ]:
#for a in ControlModelAllDataDf.columns:
    #print(a)

## Create Testing and Training data

In [ ]:
print(len(goldStandardPersons))
print(len(machineLearningDf))

In [ ]:
#We need to remove gold standard persons using the person list from the gold standard cohort
#If there's not enough genetic data to train on, we can split up the gold standard cohort and just reserve a part of it for later
#Alter the code below....

def makeTestAndTrainSets(df, trainProp):
    #First, control for the df being in the n3c cohort
    
    df = df[df["person_id"].isin(n3cCohort['person_id'])]
    
    trainDf = df[df['long_covid']==0].sample(frac=trainProp, random_state=101)
    trainDf =pd.concat([trainDf,df[df['long_covid']==1].sample(frac=trainProp, random_state=101)], ignore_index=True)
    
    NotInList=[]
    
    testDf =  df


    toDropList = df.index[df['person_id'].isin(trainDf['person_id'].tolist() + NotInList)]

    #print(toDropList)
    testDf = testDf.drop(index = toDropList, axis=0)
    testDf = testDf.reset_index(drop=True)
    return trainDf, testDf
    #trainDf=trainDf.drop(['min_long_covid_date'], axis=1)

In [ ]:
#Apply the function twice, once to make a gold standard cohort, and the other to make training and testing data
machineLearningDf, goldStandardDf = makeTestAndTrainSets(machineLearningDf, 0.7)

In [ ]:
#If we need to remove extra pieces from the training or test sets, we can use this list
trainDf, testDf = makeTestAndTrainSets(machineLearningDf, 0.5/0.7)

In [ ]:
#We need to make a test cohort of people with both survey and mobile device data
XGBtestSet=outerJoinSurveyMobileDevice
XGBtestSet =  XGBtestSet[XGBtestSet['person_id'].isin(goldStandardDf['person_id'])]

In [ ]:
len(outerJoinSurveyMobileDevice) - len(XGBtestSet)

In [ ]:
#Apply the simplifying assumption to the test and training data, but not the gold standard set
trainDf = simplifyingAssumption(trainDf)
testDf = simplifyingAssumption(testDf)

In [ ]:
#goldStandardDf

### create training and test data for the control model (all data)

In [ ]:
#remove anywhere where longcovid is null or NA
ControlModelAllDataDf = ControlModelAllDataDf.dropna(subset = ['person_id', 'long_covid'])

In [ ]:
#ControlModelAllDataDf
#first take out the gold standard cohort, then make training and testing data

GoldStandardControl = ControlModelAllDataDf[ControlModelAllDataDf['person_id'].isin(goldStandardDf['person_id'])]

all_list = list(ControlModelAllDataDf['person_id'])
remove_list = list(goldStandardDf['person_id'])
nonGoldControls = [i for i in all_list if i not in remove_list]

print(len(nonGoldControls))
print(len(ControlModelAllDataDf))

In [ ]:
nonGoldControlDf = ControlModelAllDataDf[ControlModelAllDataDf['person_id'].isin(nonGoldControls)]
trainControlDf, testControlDf = makeTestAndTrainSets(nonGoldControlDf, 0.5/0.7)

In [ ]:
GoldStandardControl = GoldStandardControl.reset_index(drop=True)
GoldStandardControl

In [ ]:
#Something wrong with this evaluation set, because it has 400 extra points and turns up bad results
GoldStandardCohort = getFromBucket('GoldStandardAllData.csv')

insertColumn = machineLearningDf['chr6:41515652:G:C_C']
GoldStandardCohort = GoldStandardCohort.join(insertColumn, how ='inner')
#GoldStandardCohort = GoldStandardCohort
#GoldStandardCohort = removeNonNumeric(GoldStandardCohort)

#We need to take out NAN values for the new gene and re-run

In [ ]:
#For some reason, there is no overlap in person_id's from the old cohort and the new cohort
#figure out why this is

GoldStandardCohort['person_id']

#GoldStandardControl['person_id']


In [ ]:
#storeInBucket(machineLearningDf, a = 'GeneticDataDfAugust15.csv')
storeInBucket(GoldStandardCohort, a = 'GoldStandardAllDataAugust15.csv')

## Create the Models 

In [ ]:
def trainModel(train, test, predictorVars,targetVar, model):
    x=train.loc[:,predictorVars]
    y=train[targetVar]
    xTest=test.loc[:,predictorVars]
    yTest=test[targetVar]

    regressor = model
    regressor.fit(x, y) 
    
    print("R-Squared Train:",metrics.r2_score(y, regressor.predict(x)))
    print("R-Squared Test:",metrics.r2_score(yTest, regressor.predict(xTest)))

In [ ]:
def generatePredValues(train, test, model, predVars):
    
    regressor = model
    regressor.fit(train.loc[:,predVars], train[target])
    averageValues = []
    nonRounded = []
    predictions = regressor.predict(test.loc[:,predVars])
    #print(len(testDf))
    
    for i in range(0,len(testDf)):
        modelPred = predictions[i]
        
        if float(modelPred) <=0:
            modelPred = 0
        if float(modelPred) >=1:
            modelPred = 1
        roundNum  = round(modelPred,0)
        averageValues = averageValues +[roundNum]
        nonRounded = nonRounded +[modelPred]
        
    predValues=averageValues
        
    return predValues, nonRounded

In [ ]:
def weightedAverage(trainDf, testDf, model,predictorVariables, p):
    q = 1-p
    regressor = model
    regressor.fit(trainDf.loc[:,predictorVariables], trainDf[target])
    averageValues = []
    nonRounded = []
    predictions = regressor.predict(testDf.loc[:,predictorVariables])
    #print(len(testDf))
    
    for i in range(0,len(testDf)):
        modelPred = predictions[i]
        weightedAverage = (q*(modelPred)+p*(testDf.loc[i,'y_pred']))
        
        if float(weightedAverage) <= 0:
            weightedAverage = 0
        if float(weightedAverage) >=1:
            weightedAverage = 1
            
        roundNum = round(weightedAverage,0)
        
        averageValues = averageValues +[roundNum]
        nonRounded = nonRounded + [weightedAverage]
        
    predValues=averageValues
        
    return predValues, nonRounded

In [ ]:
def triWeightedAverage(trainDf, testDf, model1, model2,genePredictorVariables, surveyPredictorVariables, p,q):
    x = 1-(p+q)
    regressor1 = model1
    regressor1.fit(trainDf.loc[:,genePredictorVariables], trainDf[target])
    
    regressor2 = model2
    regressor2.fit(trainDf.loc[:,surveyPredictorVariables], trainDf[target])
    
    averageValues = []
    nonRounded = []
    genePredictions = regressor1.predict(testDf.loc[:,genePredictorVariables])
    surveyPredictions = regressor2.predict(testDf.loc[:,surveyPredictorVariables])
    #print(len(testDf))
    
    for i in range(0,len(testDf)):
        genePred = genePredictions[i]
        surveyPred = surveyPredictions[i]
        weightedAverage = (p*(genePred)+q*(surveyPred)+x*(testDf.loc[i,'y_pred']))
        
        if float(weightedAverage) <= 0:
            roundNum = 0
        if float(weightedAverage) >=1:
            roundNum = 1
        roundNum = round(weightedAverage,0)
            
        averageValues = averageValues +[roundNum]
        nonRounded = nonRounded +[weightedAverage]
    predValues=averageValues
        
    return predValues, nonRounded

In [ ]:
#title = 'Confusion Matrix of EHR and Genetic Data Classifier'

def generateConfusionMatrix(test, predValues, title):
    confusionMatrix = metrics.confusion_matrix(test['long_covid'], predValues)
    
    print('Precision = ' + str(
        (confusionMatrix[1,1])
        /(confusionMatrix[1,1] + confusionMatrix[1,0] )))
          
    print('Type 1 Error = ' + str(
        confusionMatrix[0,1] / (confusionMatrix[0,0]+ confusionMatrix[0,1])))
          
    print('Type 2 Error = ' + str(
        confusionMatrix[1,0] / (confusionMatrix[1,1]+ confusionMatrix[1,0])))
    
    print('Sensitivity = ' + str(
        confusionMatrix[1,1] / (confusionMatrix[1,1]+ confusionMatrix[0,1])))
        
    print('Specificity = ' + str(
        confusionMatrix[0,0] / (confusionMatrix[1,0]+ confusionMatrix[0,0])))
          
    cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusionMatrix, display_labels = [False, True])
    cm_display.plot()
    plt.title(title)
    plt.xlabel('Predicted Long Covid Status')
    plt.ylabel('True Long Covid Status')
    plt.show()
    

In [ ]:
predVars = list(trainDf.columns)
for a in ['y_pred', 'person_id', 'long_covid']:
    predVars.remove(a)
target = 'long_covid'
#predVars

In [ ]:
surveyPredVars = predVars +[]
genePredVars = predVars+[]

for a in list(geneAppendDf.columns):
    try:
        surveyPredVars.remove(a)
    except:
        #do nothing
        notImportant=1
        
for a in list(surveyAppendDf.columns):
    try:
        genePredVars.remove(a)
    except:
        #do nothing
        notImportant=1


In [ ]:
#EHR based model performance

In [ ]:
roundedList = []
for entry in goldStandardDf['y_pred']:
    if entry >=1:
        a = 1
    else:
        a = entry
    roundedList = roundedList +[int(round(a,0))]
    
#tHIS FUNCTION IS MESSING UP A LOT, WE'LL GO BACK AND CHECK ON IT LATER
generateConfusionMatrix(goldStandardDf, roundedList, 'EHR Data Classifier Model performance')

### Linear Rergression 

In [ ]:
#predVars

In [ ]:
from sklearn import linear_model
from sklearn import metrics

trainModel(trainDf, testDf, predVars,target, linear_model.LinearRegression())

In [ ]:
predValues = generatePredValues(trainDf, testDf, linear_model.LinearRegression(), genePredVars)

In [ ]:
title = 'Confusion Matrix of Linear Model Genetic Data Classifier'
generateConfusionMatrix(testDf, predValues[0], title)

In [ ]:
# Now let's include a Confusion Matrix with the weighted average
predValues = weightedAverage(trainDf, testDf, 
                                linear_model.LinearRegression(),
                                genePredVars, 0.8)
title = 'Confusion Matrix of Genetic and EHR Data Classifier'
generateConfusionMatrix(testDf, predValues[0], title)

### Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor

trainModel(trainDf, testDf, predVars,target, RandomForestRegressor(n_estimators=50, random_state=0))

In [ ]:
#Let's try giving it more iterations to work with

trainModel(trainDf, testDf, predVars,target, RandomForestRegressor(n_estimators=1000, random_state=0))

In [ ]:
#Doesn't seem to be getting any better on our testing data, let's restrict tree complexity for overfitting

trainModel(trainDf, testDf, predVars,target, RandomForestRegressor(n_estimators=500, max_depth = 6, min_samples_split = 10, random_state=0))

In [ ]:
#More general, but not necisarrily better. Let's take a look at that confusion matrix
predValues = generatePredValues(trainDf, testDf, 
                                RandomForestRegressor(n_estimators=500, random_state=0),
                                genePredVars)
title = 'Confusion Matrix of RF Model Genetic Data Classifier'
generateConfusionMatrix(testDf, predValues[0], title)

In [ ]:
# Now let's include a Confusion Matrix with the weighted average
predValues = weightedAverage(trainDf, testDf, 
                                RandomForestRegressor(n_estimators=500, random_state=0),
                                surveyPredVars, 0.8)
title = 'Confusion Matrix of Genetic and EHR Data Classifier'
generateConfusionMatrix(testDf, predValues[0], title)

### Nueral Network

In [ ]:
from sklearn.neural_network import MLPRegressor

trainModel(trainDf, testDf, predVars,target, MLPRegressor(random_state=0))

In [ ]:
#Great on training, not good on testing. Let's limit iterations

trainModel(trainDf, testDf, predVars,target, MLPRegressor(random_state=0, n_iter_no_change = 50, 
                                              hidden_layer_sizes = (550,)
                                             ,max_iter = 500))

In [ ]:
#It's getting better with out training data, but worse with out testing

#More general, but not necisarrily better. Let's take a look at that confusion matrix
predValues = generatePredValues(trainDf, testDf, 
                                 MLPRegressor(random_state=0, n_iter_no_change = 50, 
                                              hidden_layer_sizes = (550,)
                                             ,max_iter = 500),
                                genePredVars)
title = 'Confusion Matrix of NN Model Genetic Data Classifier'
generateConfusionMatrix(testDf, predValues[0], title)

In [ ]:
# Now let's include a Confusion Matrix with the weighted average
predValues = weightedAverage(trainDf, testDf, 
                                MLPRegressor(random_state=0, n_iter_no_change = 3, hidden_layer_sizes = (500,)),
                                surveyPredVars, 0.67)
title = 'Confusion Matrix of Genetic and EHR Data Classifier'
generateConfusionMatrix(testDf, predValues[0], title)

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

trainModel(trainDf, testDf, predVars,target, KNeighborsRegressor(n_neighbors=3))

In [ ]:
#n=3 seems to be the best

#More general, but not necisarrily better. Let's take a look at that confusion matrix
genePredValues = generatePredValues(trainDf, testDf, 
                                 KNeighborsRegressor(n_neighbors=3),
                                genePredVars)
title = 'Confusion Matrix of KNN Model Genetic Data Classifier'
generateConfusionMatrix(testDf, genePredValues[0], title)

In [ ]:
# Now let's include a Confusion Matrix with the weighted average
predValues = weightedAverage(trainDf, testDf, 
                                KNeighborsRegressor(n_neighbors=3),
                                genePredVars, 0.73)
title = 'Confusion Matrix of Survey and EHR Data Classifier'
generateConfusionMatrix(testDf, predValues[0], title)

## XGBoost model (survey + mobile device data)

In [ ]:
#light gbm model needs a specific test and training set, created here
personList = list(outerJoinSurveyMobileDevice['person_id'])

for a in list(outerJoinSurveyMobileDevice['person_id']):
    if a in list(XGBtestSet['person_id']):
        personList.remove(a)


NonGoldStandard = outerJoinSurveyMobileDevice[outerJoinSurveyMobileDevice['person_id'].isin(personList)]
NonGoldStandard = NonGoldStandard.reset_index(drop=True)
NonGoldStandard = simplifyingAssumption(NonGoldStandard, 0.93)
withNaTrain, withNaTest = makeTestAndTrainSets(NonGoldStandard, 0.7)

print(len(personList))
#Reset these using XGBtestSet as the gold standard test set

In [ ]:
#NonGoldStandard.columns

In [ ]:
withNaTrain=withNaTrain.drop('person_id', axis=1).reset_index(drop=True)
withNaTest=withNaTest.reset_index(drop=True)

In [ ]:
import xgboost as xgb
yTrain = withNaTrain['long_covid']
yTest = withNaTest['long_covid']

print(len(yTrain))
print(len(yTest))

xTrain = withNaTrain.loc[:, withNaTrain.columns !='long_covid']
xTest = withNaTest.loc[:, withNaTest.columns != 'long_covid']

xTrain = xTrain.loc[:, xTrain.columns !='y_pred']
xTest = xTest.loc[:, xTest.columns != 'y_pred']
xTest = xTest.loc[:, xTest.columns != 'person_id']

In [ ]:
train = xgb.DMatrix(xTrain, label = yTrain)
test = xgb.DMatrix(xTest, label = yTest)

In [ ]:
#yTrain

In [ ]:
param = {'max_dpeth': 5,
        'eta':0.6,
        'objective': 'multi:softmax',
        'num_class':3}
epochs = 250

In [ ]:
model=xgb.train(param, train, epochs)

In [ ]:
predXGB = model.predict(test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(yTest, predXGB)

In [ ]:
#create confusion Matix
generateConfusionMatrix(withNaTest, predXGB, 'XGBoost Model Using Mobile Device and Survey Data')

## XGBoost Control Model

In [ ]:
trainControlDf=trainControlDf.drop('min_covid_dt', axis=1).reset_index(drop=True)
testControlDf=testControlDf.drop('min_covid_dt', axis=1).reset_index(drop=True)

In [ ]:
import xgboost as xgb
yTrain = trainControlDf['long_covid']
yTest = testControlDf['long_covid']

print(len(yTrain))
print(len(yTest))

xTrain = trainControlDf.loc[:, trainControlDf.columns !='long_covid']
xTest = testControlDf.loc[:, testControlDf.columns != 'long_covid']

xTrain = xTrain.loc[:, xTrain.columns !='y_pred']
xTest = xTest.loc[:, xTest.columns != 'y_pred']
#xTest = xTest.loc[:, xTest.columns != 'person_id']

In [ ]:
trainControl = xgb.DMatrix(xTrain, label = yTrain)
testControl = xgb.DMatrix(xTest, label = yTest)

In [ ]:
param = {'max_dpeth': 5,
        'eta':0.6,
        'objective': 'multi:softmax',
        'num_class':3}
epochs = 250

In [ ]:
controlModel=xgb.train(param, trainControl, epochs)

In [ ]:
predXGBControl = controlModel.predict(testControl)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(yTest, predXGBControl)

In [ ]:
#create confusion Matix
generateConfusionMatrix(testControlDf, predXGBControl, 'XGBoost Model Using Mobile Device and Survey Data')

## Using Gold Standard Cohort to combine survey data and EHR

In [ ]:
#The code below creates it's own gold standard test set
#We created what can function as a gold standard test set using XGBtestSet, so let's use that instead
#alter the code below....
withNaTest = XGBtestSet


In [ ]:
#clean up that test set
withNaTest = withNaTest[withNaTest['person_id'].isin(goldStandardDf['person_id'])]
withNaTest = withNaTest.sort_values('person_id')
withNaTest = withNaTest.reset_index(drop=True)

goldStandardDf = goldStandardDf.sort_values('person_id').reset_index(drop=True)

In [ ]:
withNaTest = withNaTest.drop('person_id', axis=1)

In [ ]:
yTest = withNaTest['long_covid']
print(len(yTest))
xTest = withNaTest.loc[:, withNaTest.columns != 'long_covid']
xTest = xTest.loc[:, xTest.columns != 'y_pred']
test = xgb.DMatrix(xTest, label = yTest)

In [ ]:
#We need to combine this model with genetic and EHR data models
#use some of the weighted trinary function to get the pred values
# Define NAtestDf = with a data frame that includes genetic and survey and mobile device data

#The following SHOULD return the same results under these parameters as the genetic and EHR data based model
#
genePredictorVariables = genePredVars

XGBTestDf = test
PandasTestDf = goldStandardDf.reset_index(drop=True)
p = 0.
q = 0.33

x = 1-(p+q)

regressorXGB= xgb.train(param, train, epochs)

    
regressorGene = RandomForestRegressor(n_estimators=500, random_state=0)
regressorGene.fit(trainDf.loc[:,genePredictorVariables], trainDf['long_covid'])
    
averageValues = []
genePredictions = regressorGene.predict(PandasTestDf.loc[:,genePredictorVariables])
surveyPredictions =  regressorXGB.predict(XGBTestDf)
#print(len(testDf))
    
for i in range(0,len(PandasTestDf)):
    genePred = genePredictions[i]
    surveyPred = surveyPredictions[i]
    weightedAverageXGB = (p*(genePred)+q*(surveyPred)+x*(PandasTestDf.loc[i,'y_pred']))
    #roundNum = round(weightedAverage,0)
    roundNum = weightedAverageXGB
    if float(roundNum) <= 0:
        roundNum = 0
    if float(roundNum) >=1:
        roundNum = 1
    averageValues = averageValues +[roundNum]
    
predValuesXGBNotRounded=averageValues

predValuesXGB = []
for a in predValuesXGBNotRounded:
    predValuesXGB = predValuesXGB + [round(a,0)]

In [ ]:
generateConfusionMatrix(withNaTest, predValuesXGB, 'All 3 models using Gold Standard Cohort')

## Using Gold Standard Cohort to mix survey, Mobile Device, EHR, and genetic data

In [ ]:
#We need to combine this model with genetic and EHR data models
#use some of the weighted trinary function to get the pred values
# Define NAtestDf = with a data frame that includes genetic and survey and mobile device data

#The following SHOULD return the same results under these parameters as the genetic and EHR data based model
#
genePredictorVariables = genePredVars

XGBTestDf = test
PandasTestDf = goldStandardDf.reset_index(drop=True)
#p = 0.1
#q = 0.4
p = 0
q = 0


x = 1-(p+q)

regressorXGB= xgb.train(param, train, epochs)

    
regressorGene =  MLPRegressor(random_state=0, n_iter_no_change = 50, 
                                              hidden_layer_sizes = (550,)
                                             ,max_iter = 500)
regressorGene.fit(trainDf.loc[:,genePredictorVariables], trainDf['long_covid'])
    
averageValues = []
genePredictions = regressorGene.predict(PandasTestDf.loc[:,genePredictorVariables])
surveyPredictions =  regressorXGB.predict(XGBTestDf)
#print(len(testDf))
    
for i in range(0,len(PandasTestDf)):
    genePred = genePredictions[i]
    surveyPred = surveyPredictions[i]
    weightedAverageXGB = (p*(genePred)+q*(surveyPred)+x*(PandasTestDf.loc[i,'y_pred']))
    #roundNum = round(weightedAverage,0)
    roundNum = weightedAverageXGB
    if float(roundNum) <= 0:
        roundNum = 0
    if float(roundNum) >=1:
        roundNum = 1
    averageValues = averageValues +[roundNum]
    
predValuesXGBNotRounded=averageValues

predValuesXGB = []
for a in predValuesXGBNotRounded:
    predValuesXGB = predValuesXGB + [round(a,0)]

In [ ]:
generateConfusionMatrix(withNaTest, predValuesXGB, 'All 3 models using Gold Standard Cohort')

# Add them all together to find the best model

In [ ]:
allDataPredValues = triWeightedAverage(trainDf, goldStandardDf, 
                    MLPRegressor(random_state=0, n_iter_no_change = 3, hidden_layer_sizes = (500,)),
                   MLPRegressor(random_state=0),
                   genePredVars, surveyPredVars,
                   .17,.16)

title = 'Confusion Matrix of Independant Working Models Classifier'
generateConfusionMatrix(goldStandardDf, allDataPredValues[0], title)

## Determine optimal Weights

In [ ]:
def getAUROCScore(predValues, testy):
    
    lr_probs = predValues
    # keep probabilities for the positive outcome only
    #lr_probs = lr_probs[:, 1]
    
    # calculate scores
    lr_auc = roc_auc_score(testy, lr_probs)
    
    #print summarized score
    #print(': ROC AUC=%.3f' % (lr_auc))
    
    return lr_auc

In [ ]:
#Define a function that given weights, returns an AUROC score
def generateWeightedAUROCScore(p, q, survey_model, gene_model, survey_testing_df, gene_testing_df):
    genePredictorVariables = genePredVars

    XGBTestDf = survey_testing_df
    PandasTestDf = gene_testing_df.reset_index(drop=True)

    x = 1-(p+q)

    regressorXGB= survey_model
    regressorGene =  gene_model
    
    averageValues = []
    genePredictions = regressorGene.predict(gene_testing_df.loc[:,genePredictorVariables])
    surveyPredictions =  regressorXGB.predict(survey_testing_df)
    #print(len(testDf))

    for i in range(0,len(PandasTestDf)):
        genePred = genePredictions[i]
        surveyPred = surveyPredictions[i]
        weightedAverageXGB = (p*(genePred)+q*(surveyPred)+x*(PandasTestDf.loc[i,'y_pred']))
        roundNum = weightedAverageXGB
        if float(roundNum) <= 0:
            roundNum = 0
        if float(roundNum) >=1:
            roundNum = 1
        averageValues = averageValues +[roundNum]

    predValuesXGBNotRounded=averageValues
    
    #Use predValues to get AUROC Score

    testy = PandasTestDf['long_covid']
    auroc_score = getAUROCScore (predValuesXGBNotRounded, testy)
    return auroc_score

In [ ]:
#Create lists with posible weights
p_weight_list = [num / 100 for num in range(0, 50, 10)]
q_weight_list = p_weight_list
weight_len = len(p_weight_list) * len(q_weight_list)

#Now create a dataframe full of zero's that we can fill in
weight_df = pd.DataFrame(np.zeros((weight_len, 3)), columns = ['p', 'q', 'AUROC_Score'])

counter = 0
for p in p_weight_list:
    for q in q_weight_list:
        weight_df.loc[counter,'p'] = p
        weight_df.loc[counter,'q'] = q
        weight_df.loc[counter,'AUROC_Score'] = generateWeightedAUROCScore(p, q, regressorXGB, regressorGene, XGBTestDf, PandasTestDf)
        counter = counter+1

In [ ]:
#PandasTestDf['y_pred'].unique()
#getAUROCScore(PandasTestDf['y_pred'], PandasTestDf['long_covid'])
#[num / 100 for num in range(0, 50, 10)]

In [ ]:
weight_df.sort_values('AUROC_Score', ascending=False)
#We see that the best performing are ~ p=0.1 and q=0.3
#Now we do it again, but this time we test everything between p=0.0 -> 0.2, and q= 0.2-> 0.4 

In [ ]:
#Create lists with posible weights
p_weight_list = [(0.1 - ((num-10) / 100)) for num in range(0, 20, 4)]
q_weight_list = [(0.4 - ((num-10) / 100)) for num in range(0, 20, 4)]
weight_len = len(p_weight_list) * len(q_weight_list)

#Now create a dataframe full of zero's that we can fill in
weight_df = pd.DataFrame(np.zeros((weight_len, 3)), columns = ['p', 'q', 'AUROC_Score'])

counter = 0
for p in p_weight_list:
    for q in q_weight_list:
        weight_df.loc[counter,'p'] = p
        weight_df.loc[counter,'q'] = q
        weight_df.loc[counter,'AUROC_Score'] = generateWeightedAUROCScore(p, q, regressorXGB, regressorGene, XGBTestDf, PandasTestDf)
        counter = counter+1

In [ ]:
weight_df.sort_values('AUROC_Score', ascending=False)

In [ ]:
#Do it one more time, but now just to see if it is +- 0.02

#Create lists with posible weights
p_weight_list = [(0.12 - ((num-7) / 100)) for num in range(0, 15, 1)]
q_weight_list = [(0.34 - ((num-7) / 100)) for num in range(0, 15, 1)]
weight_len = len(p_weight_list) * len(q_weight_list)

#Now create a dataframe full of zero's that we can fill in
weight_df = pd.DataFrame(np.zeros((weight_len, 3)), columns = ['p', 'q', 'AUROC_Score'])

counter = 0
for p in p_weight_list:
    for q in q_weight_list:
        weight_df.loc[counter,'p'] = p
        weight_df.loc[counter,'q'] = q
        weight_df.loc[counter,'AUROC_Score'] = generateWeightedAUROCScore(p, q, regressorXGB, regressorGene, XGBTestDf, PandasTestDf)
        counter = counter+1

In [ ]:
weight_df.sort_values('AUROC_Score', ascending=False)

# Data Visualization: AUROC Curves

In [ ]:
#We probably need to give the function un-rounded predValues for it to work right


# roc curve and auc


def createAUROC (predValues, testy, modelLabel):
    
    # generate a no skill prediction (majority class)
    ns_probs = [0 for _ in range(len(testy))]
    
    lr_probs = predValues
    # keep probabilities for the positive outcome only
    #lr_probs = lr_probs[:, 1]
    
    # calculate scores
    ns_auc = roc_auc_score(testy, ns_probs)
    lr_auc = roc_auc_score(testy, lr_probs)
    
    # summarize scores
    print('No Skill: ROC AUC=%.3f' % (ns_auc))
    print(modelLabel+': ROC AUC=%.3f' % (lr_auc))
    
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(testy, ns_probs)
    lr_fpr, lr_tpr, _ = roc_curve(testy, lr_probs)
    
    # plot the roc curve for the model
    pyplot.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill')
    pyplot.plot(lr_fpr, lr_tpr, marker='.', label=modelLabel)
    
    # axis labels
    pyplot.xlabel('1 - Specificity')
    pyplot.ylabel('Sensitivity')
    
    # show the legend
    pyplot.legend()
    
    # show the plot
    pyplot.show()
    return lr_auc

In [ ]:
# All 3 models with gold standard after introduction of mobile device data
#This is weighed the same as the last model, but is returning worse results. Find out why
#0.678 (p = 0.1, q = 0.4)
createAUROC(predValuesXGBNotRounded, withNaTest['long_covid'], 'All models using Gold Standard Cohort')

In [ ]:
# All 3 models without mobile device data
a = createAUROC(allDataPredValues[1], goldStandardDf['long_covid'], 'EHR, Genetic, and Survey')
a

In [ ]:
#Survey and EHR data only (KNN)

surveyEHRPredValues = weightedAverage(trainDf, goldStandardDf, MLPRegressor(random_state=0), surveyPredVars, 0.67)

title = 'Survey and EHR Data Classifier'
#generateConfusionMatrix(testDf, predValues[0], title)

createAUROC(surveyEHRPredValues[1], goldStandardDf['long_covid'], title)

In [ ]:
#Genetic and EHR data Only (RF)
geneEHRPredValues = weightedAverage(trainDf, goldStandardDf, 
                                KNeighborsRegressor(n_neighbors=3),
                                genePredVars, 0.79)

title = 'Genetic and EHR Data Classifier'
#generateConfusionMatrix(testDf, predValues[0], title)

createAUROC(geneEHRPredValues[1], goldStandardDf['long_covid'], title)

In [ ]:
#EHR data only
title = 'EHR Data Classifier'
createAUROC(goldStandardDf['y_pred'], goldStandardDf['long_covid'], title)

In [ ]:
#Survey Data Only
surveyPredValues = weightedAverage(trainDf, goldStandardDf, MLPRegressor(random_state=0), surveyPredVars, 0)

title = 'Survey Data Classifier'
createAUROC(surveyPredValues[1], goldStandardDf['long_covid'], title)

In [ ]:
#Genetic Data Only
genePredValues = weightedAverage(trainDf, goldStandardDf, 
                                 MLPRegressor(random_state=0, n_iter_no_change = 50, 
                                              hidden_layer_sizes = (550,)
                                             ,max_iter = 500),
                                genePredVars, 0)

title = 'Genetic Data Classifier'
createAUROC(genePredValues[1], goldStandardDf['long_covid'], title)

## Control Model performance

In [ ]:
#Control Model

controlPreds =  controlModel.predict(testControl)


createAUROC(controlPreds, testControlDf['long_covid'], 'Control Model')

# Feature Importance And Conglomerate Models comparison

In [ ]:
def getAUROCValues (predValues, testy, modelLabel):
    
    lr_probs = predValues
    # keep probabilities for the positive outcome only
    #lr_probs = lr_probs[:, 1]
    
    listName = []
    # calculate scores
    lr_auc = roc_auc_score(testy, lr_probs)
    
    # summarize scores
    print(modelLabel+': ROC AUC=%.3f' % (lr_auc))
    
    # calculate roc curves
    lr_fpr, lr_tpr, _ = roc_curve(testy, lr_probs)
    returnList = roc_curve(testy, lr_probs)
    return returnList


    ## plot the roc curve for the model
    #pyplot.plot(lr_fpr, lr_tpr, marker='.', label=modelLabel, color = graphColor)
    
    ## axis labels
    #pyplot.xlabel('1 - Specificity')
    #pyplot.ylabel('Sensitivity')

In [ ]:
#Add a bunch of plots then show them all

#Single Data Sources
fpr1, tpr1, _ = getAUROCValues(goldStandardDf['y_pred'], goldStandardDf['long_covid'], 'EHR Data Classifier' )
pyplot.plot(fpr1, tpr1, marker='.', label='EHR Data Classifier', color = 'blue')

fpr1, tpr1, _ = getAUROCValues(genePredValues[1], goldStandardDf['long_covid'], 'Genetic Data Classifier' )
pyplot.plot(fpr1, tpr1, marker='.', label='Genetic Data Classifier', color = 'yellow')

fpr1, tpr1, _ = getAUROCValues(surveyPredValues[1], goldStandardDf['long_covid'], 'Survey Data Classifier' )
pyplot.plot(fpr1, tpr1, marker='.', label='Survey Data Classifier', color = 'red')

#Paired Data Sources
fpr2, tpr2, _ = getAUROCValues(geneEHRPredValues[1], goldStandardDf['long_covid'], 'Genetic and EHR Data Classifier')
pyplot.plot(fpr2, tpr2, marker='.', label='Genetic and EHR Data Classifier', color = 'green')

fpr3, tpr3, _ = getAUROCValues(surveyEHRPredValues[1], goldStandardDf['long_covid'], 'Survey and EHR Data Classifier')
pyplot.plot(fpr3, tpr3, marker='.', label='Combined Data Classifier', color = 'purple')

#All Data Sources
#fpr4, tpr4, _ = getAUROCValues(allDataPredValues[1], goldStandardDf['long_covid'], 'Survey, Genetic, and EHR Data Classifier')
#pyplot.plot(fpr4, tpr4, marker='.', label='Survey, Genetic, and EHR Data Classifier', color = 'brown')

fpr5, tpr5, _ = getAUROCValues(predValuesXGBNotRounded, withNaTest['long_covid'], 'Mobile Device, Survey, Genetic, and EHR Data Classifier')
pyplot.plot(fpr5, tpr5, marker='.', label='Mobile Device, Survey, Genetic, and EHR Data Classifier', color = 'black')


# axis labels
pyplot.xlabel('1 - Specificity')
pyplot.ylabel('Sensitivity')

# show the legend
#pyplot.legend()
    
# show the plot
pyplot.show()

In [ ]:
#If it can't find it, run "pip install shap" in terminal
import shap 

In [ ]:
#Define a function to test a model with shap values
def applyShap(model, trainDf, testDf, predVars, targetVar):
    
    model.fit(trainDf[predVars], trainDf[targetVar])
    predictionShap = model.predict(testDf[predVars])

    explainer = shap.Explainer(model.predict, testDf[predVars])
    shap_values = explainer(testDf[predVars])

    
    shap.plots.beeswarm(shap_values)
    
    return shap_values

In [ ]:
#Use SHAP for feature importance on each model
#Start with the Survey Data NN

#surveyModelShap = applyShap(MLPRegressor(random_state=0), trainDf, testDf, surveyPredVars, 'long_covid')


In [ ]:
#shap.plots.waterfall(surveyModelShap[0])

In [ ]:
#geneModelShap = applyShap( MLPRegressor(random_state=0, n_iter_no_change = 3, hidden_layer_sizes = (500,)), trainDf, testDf, genePredVars, 'long_covid')

In [ ]:
#shap.plots.waterfall(geneModelShap[0])

In [ ]:
#Find a way to do shap for XGBoost algorithm
model=xgb.train(param, train, epochs)



pred = model.predict(train, output_margin=True)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(train)


shap.summary_plot(shap_values)

In [ ]:
import xgboost

xgboost.plot_importance(model)

### Venn Diagram with data types

In [ ]:
#EHRPersonIDs = list(LongCovidEHRDf['person_id'])+list(OuterMobileData['person_id'])+list(SurveyDataDF['person_id'])+list(GeneticDataDf['person_id'])

#DataDict = {'Patients with EHR Data':set(EHRPersonIDs), 'Patients with Mobile Device Data':set(OuterMobileData['person_id']), 
            'Patients with Survey Data':set(SurveyDataDF['person_id']), 'Patients with Genetic Data':set(GeneticDataDf['person_id'])}

#from venn import venn
#venn(DataDict)

In [ ]:
#To make a pretty Venn Diagram, we can export these sets and read them in R using nVennR

MobileDevicePersons = OuterMobileData['person_id']
SurveyDataPersons = SurveyDataDF['person_id']
GeneticDataPersons = GeneticDataDf['person_id']

#popList = [MobileDevicePersons, SurveyDataPersons, GeneticDataPersons]

storeInBucket(MobileDevicePersons, "MobileDevicePersons.csv")
storeInBucket(SurveyDataPersons, "SurveyDataPersons.csv")
storeInBucket(GeneticDataPersons, "GeneticDataPersons")
storeInBucket(GoldStandardControl, "GoldStandardAllData.csv")

###### 